In [16]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 22: Monkey Map ---</h2><p>The monkeys take you on a surprisingly easy trail through the jungle. They're even going in roughly the right direction according to your handheld device's Grove Positioning System.</p>
<p>As you walk, the monkeys explain that the grove is protected by a <em>force field</em>. To pass through the force field, you have to enter a password; doing so involves tracing a specific <em>path</em> on a strangely-shaped board.</p>
<p>At least, you're pretty sure that's what you have to do; the elephants aren't exactly fluent in monkey.</p>
<p>The monkeys give you notes that they took when they last saw the password entered (your puzzle input).</p>
<p>For example:</p>
<pre><code>        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
</code></pre>

<p>The first half of the monkeys' notes is a <em>map of the board</em>. It is comprised of a set of <em>open tiles</em> (on which you can move, drawn <code>.</code>) and <em>solid walls</em> (tiles which you cannot enter, drawn <code>#</code>).</p>
<p>The second half is a description of <em>the path you must follow</em>. It consists of alternating numbers and letters:</p>
<ul>
<li>A <em>number</em> indicates the <em>number of tiles to move</em> in the direction you are facing. If you run into a wall, you stop moving forward and continue with the next instruction.</li>
<li>A <em>letter</em> indicates whether to turn 90 degrees <em>clockwise</em> (<code>R</code>) or <em><span title="Or &quot;anticlockwise&quot;, if you're anti-counterclockwise.">counterclockwise</span></em> (<code>L</code>). Turning happens in-place; it does not change your current tile.</li>
</ul>
<p>So, a path like <code>10R5</code> means "go forward 10 tiles, then turn clockwise 90 degrees, then go forward 5 tiles".</p>
<p>You begin the path in the leftmost open tile of the top row of tiles. Initially, you are facing <em>to the right</em> (from the perspective of how the map is drawn).</p>
<p>If a movement instruction would take you off of the map, you <em>wrap around</em> to the other side of the board. In other words, if your next tile is off of the board, you should instead look in the direction opposite of your current facing as far as you can until you find the opposite edge of the board, then reappear there.</p>
<p>For example, if you are at <code>A</code> and facing to the right, the tile in front of you is marked <code>B</code>; if you are at <code>C</code> and facing down, the tile in front of you is marked <code>D</code>:</p>
<pre><code>        ...#
        .#..
        #...
        ....
...#.<em>D</em>.....#
........#...
<em>B</em>.#....#...<em>A</em>
.....<em>C</em>....#.
        ...#....
        .....#..
        .#......
        ......#.
</code></pre>
<p>It is possible for the next tile (after wrapping around) to be a <em>wall</em>; this still counts as there being a wall in front of you, and so movement stops before you actually wrap to the other side of the board.</p>
<p>By drawing the <em>last facing you had</em> with an arrow on each tile you visit, the full path taken by the above example looks like this:</p>
<pre><code>        &gt;&gt;v#    
        .#v.    
        #.v.    
        ..v.    
...#...v..v#    
&gt;&gt;&gt;v...<em>&gt;</em>#.&gt;&gt;    
..#v...#....    
...&gt;&gt;&gt;&gt;v..#.    
        ...#....
        .....#..
        .#......
        ......#.
</code></pre>
<p>To finish providing the password to this strange input device, you need to determine numbers for your final <em>row</em>, <em>column</em>, and <em>facing</em> as your final position appears from the perspective of the original map. Rows start from <code>1</code> at the top and count downward; columns start from <code>1</code> at the left and count rightward. (In the above example, row 1, column 1 refers to the empty space with no tile on it in the top-left corner.) Facing is <code>0</code> for right (<code>&gt;</code>), <code>1</code> for down (<code>v</code>), <code>2</code> for left (<code>&lt;</code>), and <code>3</code> for up (<code>^</code>). The <em>final password</em> is the sum of 1000 times the row, 4 times the column, and the facing.</p>
<p>In the above example, the final row is <code>6</code>, the final column is <code>8</code>, and the final facing is <code>0</code>. So, the final password is 1000 * 6 + 4 * 8 + 0: <code><em>6032</em></code>.</p>
<p>Follow the path given in the monkeys' notes. <em>What is the final password?</em></p>
</article>


In [17]:
from copy import deepcopy

from matplotlib.colors import ListedColormap


s = """
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
"""

tests = [
    {
        "name": "Example 1",
        "s": s,
        "expected": 6032,
    },
]


class MonkeyMap(Print):
    directions = (0, 1), (1, 0), (0, -1), (-1, 0)

    def __init__(self, s: str) -> None:
        map, key = re.split(r"\n\n", s)

        self.monkey_map, self.rows, self.cols = self._create_map(map)

        self.path = [
            int(ch) if ch.isdecimal() else ch
            for ch in re.findall(rf"([RL]|[0-9]+)", key.strip())
        ]

    def walk_path(self, debug=False) -> int:
        row = min(m for m, _ in self.cols.values())
        col = self.rows[row][0]
        facing = 0

        if debug:
            direction = lambda f: [">", "v", "<", "^"][f]
            vivisted = {(row, col): direction(facing)}

        while self.monkey_map[(row, col)] != ".":
            if col > self.rows[row][1]:
                row += 1
                col = self.rows[row][0]
            else:
                col += 1

        for v in self.path:
            match v:
                case "R":
                    facing = (facing + 1) % 4
                case "L":
                    facing = (facing - 1) % 4
                case int():
                    dr, dc = self.directions[facing]
                    d = 1
                    while d <= v:
                        if (row + dr, col + dc) in self.monkey_map:
                            if self.monkey_map[(row + dr, col + dc)] == "#":
                                break
                            row += dr
                            col += dc
                        else:
                            rr, cc = self.borders(facing)(row, col)
                            if self.monkey_map[(rr, cc)] == "#":
                                break
                            row = rr
                            col = cc
                        d += 1

                        if debug:
                            vivisted[(row, col)] = direction(facing)

                    if debug:
                        print(self.str(vivisted))

        return 1_000 * (row + 1) + 4 * (col + 1) + facing

    def borders(self, facing: int) -> list[callable[[int]], int]:
        return [
            lambda r, c: (r, self.rows[r][0]),
            lambda r, c: (self.cols[c][0], c),
            lambda r, c: (r, self.rows[r][1]),
            lambda r, c: (self.cols[c][1], c),
        ][facing]

    @classmethod
    def _create_map(
        cls, map: str
    ) -> tuple[dict[tuple[int, int], str], dict[int][list[int]]]:
        map = [list(l) for l in map.splitlines() if l]
        monkey_map = {}
        rows = defaultdict(lambda: [inf, -inf])
        cols = defaultdict(lambda: [inf, -inf])

        for r, row in enumerate(map):
            for c, col in enumerate(row):
                if col != " ":
                    monkey_map[(r, c)] = col
                    rows[r][0] = min(rows[r][0], c)
                    rows[r][1] = max(rows[r][1], c)
                    cols[c][0] = min(cols[c][0], r)
                    cols[c][1] = max(cols[c][1], r)

        return monkey_map, rows, cols

    def str(self, vistited={}) -> str:
        mm = deepcopy(self.monkey_map)

        for k, v in vistited.items():
            mm[k] = v

        rows = len(self.rows)
        cols = max(m[1] for m in self.rows.values()) + 1
        mp = [[mm.get((r, c), " ") for c in range(cols)] for r in range(rows)]

        return f'{"\n".join("".join(c for c in r) for r in mp)}\n\n'

    def show(self) -> None:
        cmp = ListedColormap(["black", "LavenderBlush", "DarkSlateGray"])

        mm = deepcopy(self.monkey_map)
        rows = len(self.rows)
        cols = max(m[1] for m in self.rows.values()) + 1
        grid = [
            [{" ": 0, ".": 1, "#": 2}[mm.get((r, c), " ")] for c in range(cols)]
            for r in range(rows)
        ]

        plt.imshow(grid, cmap=cmp)
        plt.show()
        plt.close()


@test(tests=tests)
def test_part_I(s: str) -> int:
    return MonkeyMap(s).walk_path()


Test Example 1 passed, for test_part_I.
Success


In [18]:
with open("../input/day22.txt") as facing:
    puzzle = facing.read()

print(f"Part I: {MonkeyMap(puzzle).walk_path():,}")

Part I: 186,128


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>186128</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you reach the force field, you think you hear some Elves in the distance. Perhaps they've already arrived?</p>
<p>You approach the strange <em>input device</em>, but it isn't quite what the monkeys drew in their notes. Instead, you are met with a large <em>cube</em>; each of its six faces is a square of 50x50 tiles.</p>
<p>To be fair, the monkeys' map <em>does</em> have six 50x50 regions on it. If you were to <em>carefully fold the map</em>, you should be able to shape it into a cube!</p>
<p>In the example above, the six (smaller, 4x4) faces of the cube are:</p>
<pre><code>        1111
        1111
        1111
        1111
222233334444
222233334444
222233334444
222233334444
        55556666
        55556666
        55556666
        55556666
</code></pre>
<p>You still start in the same position and with the same facing as before, but the <em>wrapping</em> rules are different. Now, if you would walk off the board, you instead <em>proceed around the cube</em>. From the perspective of the map, this can look a little strange. In the above example, if you are at A and move to the right, you would arrive at B facing down; if you are at C and move down, you would arrive at D facing up:</p>
<pre><code>        ...#
        .#..
        #...
        ....
...#.......#
........#..<em>A</em>
..#....#....
.<em>D</em>........#.
        ...#..<em>B</em>.
        .....#..
        .#......
        ..<em>C</em>...#.
</code></pre>
<p>Walls still block your path, even if they are on a different face of the cube. If you are at E facing up, your movement is blocked by the wall marked by the arrow:</p>
<pre><code>        ...#
        .#..
     <em>--&gt;#</em>...
        ....
...#..<em>E</em>....#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.
</code></pre>
<p>Using the same method of drawing the <em>last facing you had</em> with an arrow on each tile you visit, the full path taken by the above example now looks like this:</p>
<pre><code>        &gt;&gt;v#    
        .#v.    
        #.v.    
        ..v.    
...#..<em>^</em>...v#    
.&gt;&gt;&gt;&gt;&gt;^.#.&gt;&gt;    
.^#....#....    
.^........#.    
        ...#..v.
        .....#v.
        .#v&lt;&lt;&lt;&lt;.
        ..v...#.
</code></pre>
<p>The final password is still calculated from your final position and facing from the perspective of the map. In this example, the final row is <code>5</code>, the final column is <code>7</code>, and the final facing is <code>3</code>, so the final password is 1000 * 5 + 4 * 7 + 3 = <code><em>5031</em></code>.</p>
<p>Fold the map into a cube, <em>then</em> follow the path given in the monkeys' notes. <em>What is the final password?</em></p>
</article>


The following list specifies the top, left, right, and bottom numbers for each main face of a standard right-handed die (the most common in the West). Imagine you are looking directly at the "Front" face, with the die oriented vertically:
If the Front face is 1: The number 6 is on the back (opposite). The top number is 4, the bottom is 3, the left is 2, and the right is 5.
If the Front face is 2: The number 5 is on the back (opposite). The top number is 1, the bottom is 6, the left is 4, and the right is 3.
If the Front face is 3: The number 4 is on the back (opposite). The top number is 1, the bottom is 6, the left is 5, and the right is 2.
If the Front face is 4: The number 3 is on the back (opposite). The top number is 1, the bottom is 6, the left is 2, and the right is 5.
If the Front face is 5: The number 2 is on the back (opposite). The top number is 1, the bottom is 6, the left is 3, and the right is 4.
If the Front face is 6: The number 1 is on the back (opposite). The top number is 3, the bottom is 4, the left is 2, and the right is 5.
The exact orientation of the numbers or pips themselves can vary further (e.g., whether the '2' pips are diagonal or vertical), but the relative position of the faces remains consistent for a given handedness.


In [ ]:
from collections import deque
from collections.abc import Iterator
from dataclasses import astuple
from pprint import pprint


s = """
        ...#
        .#..
        #...
        ....
...#.......#
........#...
..#....#....
..........#.
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5
"""

tests = [
    {
        "name": "Example 1",
        "s": s,
        "expected": 5031,
    },
]

type State = tuple[int, int, int, Dice]


@dataclass(eq=True, frozen=True)
class Dice:
    # 🎲
    top: int = 5
    bottom: int = 2
    front: int = 1
    back: int = 6
    left: int = 4
    right: int = 3

    def go_up(self) -> Dice:
        return Dice(
            top=self.back,
            bottom=self.front,
            front=self.top,
            back=self.bottom,
            left=self.left,
            right=self.right,
        )

    def go_down(self) -> Dice:
        return Dice(
            top=self.front,
            bottom=self.back,
            front=self.bottom,
            back=self.top,
            left=self.left,
            right=self.right,
        )

    def go_left(self) -> Dice:
        return Dice(
            top=self.top,
            bottom=self.bottom,
            front=self.right,
            back=self.left,
            left=self.front,
            right=self.back,
        )

    def go_right(self) -> Dice:
        return Dice(
            top=self.top,
            bottom=self.bottom,
            front=self.left,
            back=self.right,
            left=self.back,
            right=self.front,
        )

    def rotate_90_degress_axes_front_back(self) -> Dice:
        return Dice(
            top=self.left,
            bottom=self.right,
            front=self.front,
            back=self.back,
            left=self.bottom,
            right=self.top,
        )

    def next_dices(self) -> Iterator[Dice]:
        yield self.go_up()
        yield self.go_down()
        yield self.go_left()
        yield self.go_right()


class MonkeyMapCube(MonkeyMap):
    def __init__(self, s: str, debug=False) -> None:
        super().__init__(s)
        self.faces, self.faces_state = self._map_on_dice(debug)
        self.size = len(self.faces[1])

    def walk_path(self, debug=False) -> int:
        dice = Dice()
        row, col, facing = 0, 0, 0

        if debug:
            direction = lambda f: [">", "v", "<", "^"][f]
            visited = {(row, col, dice): direction(facing)}

        for v in self.path:
            match v:
                case "R":
                    facing = (facing + 1) % 4
                case "L":
                    facing = (facing - 1) % 4
                case int():
                    dr, dc = self.directions[facing]

                    d = 1

                    while d <= v:
                        if 0 <= row + dr < self.size and 0 <= col + dc < self.size:
                            if self.faces[dice.front][row + dr][col + dc] == "#":
                                break

                            row += dr
                            col += dc
                        else:
                            rr, cc, ff, dd = self.move_to_other_face(
                                row, col, facing, dice
                            )

                            if self.faces[dd.front][rr][cc] == "#":
                                break

                            row = rr
                            col = cc
                            facing = ff
                            dice = dd
                            dr, dc = self.directions[facing]

                        d += 1

                        if debug:
                            visited[(row, col, dice)] = direction(facing)

            if debug:
                print("In loop")
                print(f"{v=}, {row=}, {col=}, {facing=}, and {dice=}")
                print(self.str(visited))
                visited[(row, col, dice)] = direction(facing)

        if debug:
            print("End state")
            print(f"{v=}, {row=}, {col=}, {facing=}, and {dice=}")
            print(self.faces_state[dice.front])
            print(self.str(visited))

        dr, dc, _ = self.faces_state[dice.front]
        return 1_000 * (row + dr + 1) + 4 * (col + dc + 1) + facing

    def str(self, vistited={}) -> str:
        mm = deepcopy(self.monkey_map)

        for k, v in vistited.items():
            row, col, dice = k
            dr, dc, _ = self.faces_state[dice.front]
            mm[(row + dr, col + dc)] = v

        rows = len(self.rows)
        cols = max(m[1] for m in self.rows.values()) + 1
        mp = [[mm.get((r, c), " ") for c in range(cols)] for r in range(rows)]

        return f'{"\n".join("".join(c for c in r) for r in mp)}\n\n'

    def move_to_other_face(self, row: int, col: int, facing: int, dice: Dice) -> State:
        match facing:
            case 0:  #  >
                return self.correct_rotation(row, 0, facing, dice.go_right())
            case 1:  #  v
                return self.correct_rotation(0, col, facing, dice.go_down())
            case 2:  #  <
                return self.correct_rotation(row, self.size - 1, facing, dice.go_left())
            case 3:  #  ^
                return self.correct_rotation(self.size - 1, col, facing, dice.go_up())
            case _:
                raise ValueError()

    def correct_rotation(self, row: int, col: int, facing: int, dice: Dice) -> State:
        goal_state = self.faces_state[dice.front][-1]

        while dice != goal_state:
            row, col = self.size - col - 1, row
            facing = (facing - 1) % 4
            dice = dice.rotate_90_degress_axes_front_back()

        return row, col, facing, dice

    def _map_on_dice(
        self, debug=False
    ) -> tuple[list[list[list[str]]], list[tuple[int, int, Dice]]]:
        size = 1 + min(
            min(mx - mn for mn, mx in self.rows.values()),
            min(mx - mn for mn, mx in self.cols.values()),
        )
        directions = (-size, 0), (size, 0), (0, -size), (0, size)

        faces = [None] * 7
        faces_state = [None] * 7

        if debug:
            mm_to_faces = {}

        queue = deque([(0, self.rows[0][0], Dice())])

        while queue:
            row, col, dice = queue.popleft()

            face = dice.front

            if faces[face] is not None:
                continue

            if (row, col) not in self.monkey_map:
                continue

            faces[face] = [
                [self.monkey_map[(r, c)] for c in range(col, col + size)]
                for r in range(row, row + size)
            ]

            faces_state[face] = row, col, dice

            if debug:
                for r in range(row, row + size):
                    for c in range(col, col + size):
                        mm_to_faces[(r, c)] = str(face)

            for i, next_dice in enumerate(dice.next_dices()):
                dr, dc = directions[i]
                queue.append((row + dr, col + dc, next_dice))

        if debug:
            pprint(list(enumerate(faces_state)))
            print()
            print(super().str(mm_to_faces))

        return faces, faces_state


@test(tests=tests)
def test_part_I(s: str) -> int:
    mmc = MonkeyMapCube(s)
    return mmc.walk_path()


Test Example 1 passed, for test_part_I.
Success


In [20]:
print(f"PartI II: {MonkeyMapCube(puzzle).walk_path()}")

PartI II: 34426


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>34426</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
